<a href="https://colab.research.google.com/github/vivorima/VideoTracking-LK_DenseOptFlow/blob/main/Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP de Tracking**
Rima Mahmoudi, M2 VMI

Ce document contient le code et le compte rendu.

PS: Il y aura une vidéo comme output pour voir le résultat ainsi que les frames affichées avec matplotlib

Lien du Notebook: https://colab.research.google.com/drive/1jX5ETrmzCd6i23C37yewgvjbw-dr9UCF?usp=sharing

# Lucas-Kanade Optical flow

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

%matplotlib inline


video_path = "/content/traffic.mp4"
cap = cv.VideoCapture(video_path)

# Parameters for ShiTomasi corner detection
feature_params = dict(maxCorners=150,
                      qualityLevel=0.03,
                      minDistance=7,
                      blockSize=7)

# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (150, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

# initializing video for better display
fourcc = cv.VideoWriter_fourcc(*'MP4V')
out = cv.VideoWriter('optical_flow_output.mp4', fourcc, 20.0, (old_frame.shape[1], old_frame.shape[0]))


while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1] if p1 is not None and st.any() else None
    good_old = p0[st==1] if p1 is not None and st.any() else None

    # draw the tracks
    if good_new is not None:
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

        img = cv.add(frame, mask)

        out.write(img)

        plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        plt.show()

        # update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
    else:
      print("None!!")
      break

# Release the video capture object
cap.release()
# Close the video writer
out.release()

# **Compte-Rendu sur l'Implémentation de la Méthode de Flux Optique de Lucas-Kanade avec OpenCV**

La méthode de Lucas-Kanade repose sur l'hypothèse que le flux optique est essentiellement constant dans un petit voisinage de l'image.


# **Étapes principales:**

* **Détection des Points d'Intérêt :** On utilise Shi-Tomasi pour détecter les coins (corners) dans la première image (frame). Les coins trouvés sont marqués en tant que points d'intéret, car ils serviront de points de référence pour notre tracking.

* **Calcul du Flux Optique :** On applique la méthode Lucas-Kanade pour suivre les points d'intérêt extraits à travers les images de la vidéo.

# **Paramètres pour la Détection des Coins Shi-Tomasi (feature_params) :**

 * **maxCorners:** Le nombre maximal de coins à détecter. Un nombre plus élevé augmente les chances de détecter plusieurs points d'intérêt mais ils ne seront pas tous fiables. Une valeur plus faible renvoie les angles les plus pertinants.
 * **qualityLevel:** Un paramètre indiquant la qualité minimale des coins. Des valeurs plus élevées signifient des coins de meilleure qualité, mais ils seront moins nombreux.
 * **minDistance:** La distance euclidienne minimale entre les coins détectés. Cela évite de détecter des coins trop proches les uns des autres.
 * **blockSize:** La taille de la fenêtre de voisinage utilisée pour la détection des angles. Une dimension de bloc plus large rend la détection moins sensible au bruit et permet de détecter moins de coins, mais ils seront plus fiables.

# **Paramètres pour la Méthode de Lucas-Kanade (lk_params) :**
 * **winSize:** La taille de la fenêtre de recherche. Des fenêtres plus grandes peuvent capturer des mouvements plus importants, mais peuvent être moins précises pour de petits mouvements. Les fenêtres plus petites sont plus sensibles au bruit et peuvent manquer des mouvements plus importants.
 * **maxLevel:** Le nombre de niveaux de la pyramide utilisés. La valeur 0 signifie qu'aucune couche supplémentaire n'est créée dans la pyramide d'images. Des valeurs plus élevées permettent de suivre des points sur des mouvements plus importants, mais nécessitent davantage de calculs.
 * **criteria:** Critères d'arrêt de l'algorithme de recherche.

# Dense Optical Flow

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

video_path = "/content/traffic.mp4"
cap = cv.VideoCapture(video_path)

ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

# Initialize video writer
fourcc = cv.VideoWriter_fourcc(*'MP4V')
out = cv.VideoWriter('dense_optical_flow_output.mp4', fourcc, 20.0, (frame1.shape[1], frame1.shape[0]))

# Process a set number of frames
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)

    pyr_scale = 0.5
    levels = 3
    winsize = 15
    iterations = 3
    poly_n = 5
    poly_sigma = 1.2

    flow = cv.calcOpticalFlowFarneback(prvs, next, None, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    out.write(bgr)

    # Display image using matplotlib
    plt.imshow(cv.cvtColor(bgr, cv.COLOR_BGR2RGB))
    plt.show()

    prvs = next

cap.release()
out.release()  # Close the video writer


# **Compte-Rendu sur l'Implémentation de la Méthode du Flux Optique Dense de Farneback avec OpenCV**

Contrairement au Flux Optique de Lucas-Kanade, le Flux Optique Dense calcule le mouvement pour chaque pixel.

# **Étapes Principales**

* **Initialisation des Paramètres :** Conversion de la première image en niveaux de gris et initialisation d'une image HSV pour la visualisation du flux optique.
* **Calcul du Flux Optique :** À chaque image, le script calcule le flux optique en utilisant l'algorithme de Farneback, qui est ensuite converti en magnitude et en angle pour la visualisation.

# **Paramètres de la méthode de flux optique dense de Farneback**
* **pyr_scale :** Ce paramètre définit le facteur de réduction d'échelle pour la construction de la pyramide des images. Une valeur élevée a pour conséquence la perte de certains mouvements légers, une valeur faible Conserve plus de détails.

* **Levels :** Le nombre de niveaux dans la pyramide d'images, avec une valeur élevée on Capture les mouvements sur de plus grandes distances, ce qui est utile pour suivre des objets se déplaçant rapidement ou de longues séquences comme dans notre video de traffic.


* **winsize :** avec une valeur élevée, nous augmentons la résistance au bruit et la capacité à capturer de grands mouvements, mais nous risquons d'atténuer les détails.

* **iterations :** Le nombre d'itérations à chaque niveau de la pyramide. Une valeur élevée améliore la précision du calcul du flux optique, mais augmente considérablement le temps de calcul.

* **poly_n :** La taille de la fenêtre de voisinage pour l'approximation polynomiale.